In [1]:
import pandas as pd
from slugify import slugify
import json

In [6]:
raw_data = pd.read_excel('/home/abhinav/sink/D4D/data_master.xlsx', engine='openpyxl', sheet_name="data")
indicators = pd.read_excel('/home/abhinav/sink/D4D/data_master.xlsx', engine='openpyxl', sheet_name="indicators")
dist_meta_data = {"morigaon":{"Title": "Morigaon", "About": "Morigaon Dist Description"}}
dept_meta_data = {'panchayat-and-rural-development': {'Title': "Panchayat and Rural Development", 'Desc':'Department Description', 'schemes':['mgnrega']}, 
             'public-health-engineering': {'Title': "Public Health Engineering", 'Desc':'Department Description', 'schemes':['jjm']}}
scheme_meta_data = {"mgnrega": {"Name": "Mahatma Gandhi National Rural Employment Guarantee Act (MGNREGA)", "Title": "MGNREGA", "Desc": "Desc of scheme", "dept_slug":"panchayat-and-rural-development", "logo": "/logo/mgnrega.webp"},
                    "jjm": {"Name": "Jal Jeevan Mission (JJM)", "Title": "JJM", "Desc": "Desc of scheme", "dept_slug":"public-health-engineering", "logo": "/logo/nhm.png"}}

In [9]:
indicators

,scheme_name,indicator_report,indicator_name,indicator_slug,indicator_type,indicator_name_short,indicator_selected,indicator_overview,indicator_explorer,indicator_dept,indicator_district,indicator_unit
0,MGNREGA,Category Wise Household Workers,No of jobcards applied for,nojaf,district_profile,NaN,no,no,no,no,no,number
1,MGNREGA,Category Wise Household Workers,No of jobcards issued,nojid,district_profile,NaN,no,no,no,no,no,number
2,MGNREGA,Category Wise Household Workers,No of registered SC workers,nscws,district_profile,NaN,no,no,no,no,no,number
3,MGNREGA,Category Wise Household Workers,No of registered ST workers,norsw,district_profile,NaN,no,no,no,no,no,number
4,MGNREGA,Category Wise Household Workers,"No of registered non-SC, non-ST workers",norns,district_profile,NaN,no,no,no,no,no,number
...,...,...,...,...,...,...,...,...,...,...,...,...
181,MGNREGA,Derived,Share of persondays generated for women,spgfw,district_performance,Share of persondays generated for women,yes,yes,yes,no,no,percentage
182,MGNREGA,Derived,Cumulative Expenditure on Unskilled wage as a ...,cussw,district_performance,NaN,yes,no,yes,no,no,percentage
183,MGNREGA,Derived,Cumulative Expenditure on Unskilled wage as a ...,cuswm,district_performance,NaN,yes,no,yes,no,no,percentage
184,MGNREGA,Derived,Cumulative Expenditure on materials as a share...,cmsum,district_performance,NaN,yes,no,yes,no,no,percentage


In [35]:
# merging the data and indicator files to get required rows
data = raw_data[raw_data.indicator_selected == 'yes']

def get_dept_slug(x):
    
    for key in dept_data:
        if x in dept_data[key]['schemes']:
            return key
        
def get_dept_name(x):
    
    for key in dept_data:
        if x in dept_data[key]['schemes']:
            return dept_data[key]["Title"]      
    


data['dist_page'] = data['indicator_slug'].map(lambda x: "yes" if x in ['npdgn', 'nobaw', 'tcexp'] else "no")
data['dept_page'] = data['indicator_slug'].map(lambda x: "yes" if x in ['npdgn', 'nobaw', 'tcexp'] else "no")
data['narrative_page'] = data['indicator_slug'].map(lambda x: "yes" if x in ['npdgn', 'nobaw', 'notww', 'nscjc', 'nopoe', 'nopwa'] else "no")
data['explorer_page'] = data['indicator_slug'].map(lambda x: "yes")

data['district_name'] = data['district_name'].map(lambda x: x.title())
data['block_name'] = data['block_name'].map(lambda x: x.title())
data['district_slug'] = data['district_name'].map(lambda x: slugify(x))
data['block_slug'] = data['block_name'].map(lambda x: slugify(x))
data['scheme_slug'] = data['scheme_name'].map(lambda x: slugify(x))

data['dept_slug'] = data['scheme_slug'].map(lambda x: get_dept_slug(x))
data['dept_name'] = data['scheme_slug'].map(lambda x: get_dept_name(x))

data.head()

/home/abhinav/pyprojenv/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/abhinav/pyprojenv/lib/python3.6/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,block_name,block_code,district_name,district_code,financial_year,scheme_name,indicator_report,date_of_update,indicator_name,indicator_slug,...,indicator_unit,dist_page,dept_page,narrative_page,explorer_page,district_slug,block_slug,scheme_slug,dept_name,dept_slug
1912,Tamulpur,1.0,Tamulpur,32.0,2023-2024,MGNREGA,Category Wise Household Workers,7-2023-09-34,No of SC workers with active jobcards,nscjc,...,number,no,no,yes,yes,tamulpur,tamulpur,mgnrega,Panchayat and Rural Development,panchayat-and-rural-development
1913,Nagrijuli,2.0,Tamulpur,32.0,2023-2024,MGNREGA,Category Wise Household Workers,7-2023-09-34,No of SC workers with active jobcards,nscjc,...,number,no,no,yes,yes,tamulpur,nagrijuli,mgnrega,Panchayat and Rural Development,panchayat-and-rural-development
1914,Goreswar,3.0,Tamulpur,32.0,2023-2024,MGNREGA,Category Wise Household Workers,7-2023-09-34,No of SC workers with active jobcards,nscjc,...,number,no,no,yes,yes,tamulpur,goreswar,mgnrega,Panchayat and Rural Development,panchayat-and-rural-development
1915,Kaliapani,1.0,Jorhat,18.0,2023-2024,MGNREGA,Category Wise Household Workers,7-2023-09-33,No of SC workers with active jobcards,nscjc,...,number,no,no,yes,yes,jorhat,kaliapani,mgnrega,Panchayat and Rural Development,panchayat-and-rural-development
1916,Jorhat East,2.0,Jorhat,18.0,2023-2024,MGNREGA,Category Wise Household Workers,7-2023-09-33,No of SC workers with active jobcards,nscjc,...,number,no,no,yes,yes,jorhat,jorhat-east,mgnrega,Panchayat and Rural Development,panchayat-and-rural-development


In [28]:
# calculate district aggregate values for district page 
def f(x):
    d = {}
    d['notww'] = x[x['indicator_slug'] == 'notww']['indicator_value'].sum()
    d['pdcpd'] = round(100 * (x[x['indicator_slug'] == 'npdgn']['indicator_value'].sum() / x[x['indicator_slug'] == 'cpmar']['indicator_value'].sum()), 2)
    d['aeaaw'] = round(100 * (x[x['indicator_slug'] == 'nobaw']['indicator_value'].sum() / x[x['indicator_slug'] == 'notww']['indicator_value'].sum()), 2)
    d['ppcgt'] = round(100 * (x[x['indicator_slug'] == 'nwgs3']['indicator_value'].sum() / x[x['indicator_slug'] == 'tnws3']['indicator_value'].sum()), 2)

    return pd.Series(d, index=['notww', 'pdcpd', 'aeaaw', 'ppcgt']) 

dist_data = data.groupby(['district_name', 'district_slug']).apply(f).reset_index()
dist_data.head()



/home/abhinav/pyprojenv/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in double_scalars
  """


,district_name,district_slug,notww,pdcpd,aeaaw,ppcgt
0,Bajali,bajali,75195.0,20.55,63.76,86.77
1,Baksa,baksa,121904.0,53.79,58.48,86.78
2,Barpeta,barpeta,323601.0,139.00,70.06,83.59
3,Biswanath,biswanath,176750.0,28.13,45.95,88.99
4,Bongaigaon,bongaigaon,167207.0,31.69,42.16,73.22


In [50]:
# District Page Generator

base_json = {}

for index, row in dist_data.iterrows():
    
    # insert district
    if  row.district_slug not in base_json:
        base_json[row.district_slug] = {
            "title": row.district_name,
            "href": row.district_slug,
            "collapsible": {
            "title": "District Highlights & Information",
            "content": {
                "cards": [
                    {
                        "value": str(row.notww),
                        "label": "Number of workers with active job cards",
                        "color": "highlight"
                    },
                    {
                        "value": str(row.pdcpd) + " %", #"₹" + 
                        "label": "Person Days Generated as a share of Cumulative Projection of Person Days",
                        "color": "highlight"
                    },
                    {
                        "value": str(row.aeaaw) + " %", #"₹" + 
                        "label": "ABPS Enabled Accounts as a share of Total Active Workers",
                        "color": "highlight"
                    },
                    {
                        "value": str(row.ppcgt) + " %", #"₹" + 
                        "label": "Percentage of Projects with completed geotagging",
                        "color": "highlight"
                    }
                ],
                "rightTitle": "About " + row.district_name,
                "description": [dist_meta_data[row.district_slug]["About"] if row.district_slug in dist_meta_data else row.district_name]
            }
            },
            "listTitle": "Browse Line Departments",
            "list": [
            {
                "label": "Panchayat & Rural Development",
                "href": "panchayat-and-rural-development"
            },
            {
                "label": "Public Health Engineering",
                "href": "public-health-engineering"
            }
            ]
            }
            
    else:
        pass
        
with open('dist_data.json', 'w') as fp:
    json.dump(base_json, fp, indent=4)

In [56]:
# calculate district aggregate values for schemes on department page 
def f(x):
    d = {}
    if x.iloc[0]['scheme_slug'] == "mgnrega":
        d['notww'] = x[x['indicator_slug'] == 'notww']['indicator_value'].sum()
        d['pdcpd'] = round(100 * (x[x['indicator_slug'] == 'npdgn']['indicator_value'].sum() / x[x['indicator_slug'] == 'cpmar']['indicator_value'].sum()), 2)
        d['aeaaw'] = round(100 * (x[x['indicator_slug'] == 'nobaw']['indicator_value'].sum() / x[x['indicator_slug'] == 'notww']['indicator_value'].sum()), 2)
    elif x.iloc[0]['scheme_slug'] == "jjm":
        d['notwwjjm'] = x[x['indicator_slug'] == 'notww']['indicator_value'].sum()
    else:
        pass
    
    d['last_updated'] = x.iloc[0]['date_of_update']
        
    return pd.Series(d, index=['last_updated','notww', 'pdcpd', 'aeaaw', 'notwwjjm']) 

dept_data = data.groupby(['district_name', 'district_slug', 'dept_name', 'dept_slug', 'scheme_name', 'scheme_slug']).apply(f).reset_index()
dept_data.head()


#Test data cration
# remove in production 
dept_data = dept_data.append(dept_data.iloc[0], ignore_index=True)
dept_data['dept_slug'].iloc[35]  = "public-health-engineering"
dept_data['scheme_slug'].iloc[35]  = "jjm"
dept_data['notwwjjm'].iloc[35]  = 500

/home/abhinav/pyprojenv/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in double_scalars
  


,district_name,district_slug,dept_name,dept_slug,scheme_name,scheme_slug,last_updated,notww,pdcpd,aeaaw,notwwjjm
0,Bajali,bajali,Panchayat and Rural Development,panchayat-and-rural-development,MGNREGA,mgnrega,7-2023-09-31,75195.0,20.55,63.76,NaN
1,Baksa,baksa,Panchayat and Rural Development,panchayat-and-rural-development,MGNREGA,mgnrega,7-2023-09-31,121904.0,53.79,58.48,NaN
2,Barpeta,barpeta,Panchayat and Rural Development,panchayat-and-rural-development,MGNREGA,mgnrega,7-2023-09-31,323601.0,139.00,70.06,NaN
3,Biswanath,biswanath,Panchayat and Rural Development,panchayat-and-rural-development,MGNREGA,mgnrega,7-2023-09-31,176750.0,28.13,45.95,NaN
4,Bongaigaon,bongaigaon,Panchayat and Rural Development,panchayat-and-rural-development,MGNREGA,mgnrega,7-2023-09-31,167207.0,31.69,42.16,NaN


In [100]:
# Dept Page Generator

base_json = {}

for index, row in dept_data.iterrows():
    
    # insert district
    if  row.district_slug not in base_json:
        base_json[row.district_slug] = {
        "distTitle": row.district_name,
        "depts" :
            {
                row.dept_slug: 
                   {
                        "deptTitle": row.dept_name,
                        "collapsible": {
                        "title": "Department Information",
                        "content": [
                            dept_meta_data[row.dept_slug]["Desc"] if row.dept_slug in dept_meta_data else "Dummy Dept Desc"
                            ]
                        },
                        "listTitle": "Browse Schemes and Surveys",
                        "list": [
                        {
                            "label": scheme_meta_data[row.scheme_slug]["Name"],
                            "href": row.scheme_slug,
                            "image": scheme_meta_data[row.scheme_slug]["logo"],
                            "lastUpdated": row.last_updated,
                            "cards": [
                            {
                                "value": ("INR " + str(row.notww)) if row.scheme_slug == "mgnrega" else row.notwwjjm,
                                "label": "Average Daily Wage" if row.scheme_slug == "mgnrega" else "JJM label"
                            },
                            {
                                "value": ("INR " + str(row.pdcpd))  if row.scheme_slug == "mgnrega" else row.notwwjjm,
                                "label": "Cost / PersonDay" if row.scheme_slug == "mgnrega" else "JJM label"
                            },
                            {
                                "value": (row.aeaaw)  if row.scheme_slug == "mgnrega" else row.notwwjjm,
                                "label":"Person Days Generated" if row.scheme_slug == "mgnrega" else "JJM label",
                                "type" : "progress"
                            }
                            ]
                        },
                        ]
                    },
            }
            
        }

            
    elif  row.dept_slug not in base_json[row.district_slug]["depts"]:
        
        base_json[row.district_slug]["depts"][row.dept_slug] =  {
                        "deptTitle": row.dept_name,
                        "collapsible": {
                        "title": "Department Information",
                        "content": [
                            dept_meta_data[row.dept_slug]["Desc"] if row.dept_slug in dept_meta_data else "Dummy Dept Desc"
                            ]
                        },
                        "listTitle": "Browse Schemes and Surveys",
                        "list": [
                        {
                            "label": scheme_meta_data[row.scheme_slug]["Name"],
                            "href": row.scheme_slug,
                            "image": scheme_meta_data[row.scheme_slug]["logo"],
                            "lastUpdated": row.last_updated,
                            "cards": [
                            {
                                "value": ("INR " + str(row.notww)) if row.scheme_slug == "mgnrega" else row.notwwjjm,
                                "label": "Average Daily Wage" if row.scheme_slug == "mgnrega" else "JJM label"
                            },
                            {
                                "value": ("INR " + str(row.pdcpd))  if row.scheme_slug == "mgnrega" else row.notwwjjm,
                                "label": "Cost / PersonDay" if row.scheme_slug == "mgnrega" else "JJM label"
                            },
                            {
                                "value": (row.aeaaw)  if row.scheme_slug == "mgnrega" else row.notwwjjm,
                                "label":"Person Days Generated" if row.scheme_slug == "mgnrega" else "JJM label",
                                "type" : "progress"
                            }
                            ]
                        },
                        ]
                    }
    else:
        pass

        
with open('dept_data.json', 'w') as fp:
    json.dump(base_json, fp, indent=4)